In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from subprocess import Popen, PIPE, call

In [ ]:
#loci_nt
path_all_all = '/home/juan/Desktop/juan/bio/mrcv/data/ss/all_all/'
path_all_21 = '/home/juan/Desktop/juan/bio/mrcv/data/ss/all_21nt/'
path_mite_all = '/home/juan/Desktop/juan/bio/mrcv/data/ss/mite_all/'
path_mite_21 = '/home/juan/Desktop/juan/bio/mrcv/data/ss/mite_21/'

#mirbase
path_mite_21_seqs = '/home/juan/Desktop/juan/bio/mrcv/data/ss/mite_21/mir.Y.fasta'
path_mirbase = '/home/juan/Desktop/juan/bio/mrcv/data/ss/mite_21/mir.Y.mirbase.csv'

#annotations
file_ann = '/home/juan/Desktop/juan/bio/mrcv/data/genome/annotations/introns.gff3'
file_ann_promoters = '/home/juan/Desktop/juan/bio/mrcv/data/genome/annotations/promoters.gff3'

In [ ]:
df_all_all = pd.read_csv(path_all_all + 'Results.txt', sep='\t')
print('all_all ' + len(df_all_all.index))

In [ ]:
df_all_all = df_all_all[df_all_all.MIRNA == 'Y']
print('all_all miRNA' + len(df_all_all.index))

In [ ]:
df_all_21 = pd.read_csv(path_all_21 + 'Results.txt', sep='\t')
print('all_21 ' + len(df_all_21.index))

In [ ]:
df_all_21 = df_all_21[df_all_21.MIRNA == 'Y']
print('all_21 miRNA ' + len(df_all_21.index))

In [ ]:
df_mite_all = pd.read_csv(path_mite_all + 'Results.txt', sep='\t')
print('mite_all ' + len(df_mite_all.index))

In [ ]:
df_mite_all = df_mite_all[df_mite_all.MIRNA == 'Y']
print('mite_all ' + len(df_mite_all.index))

In [ ]:
df_mite_21 = pd.read_csv(path_mite_21 + 'Results.txt', sep='\t')
print('mite_21 ' + len(df_mite_21.index))

In [ ]:
df_mite_21 = df_mite_21[df_mite_21.MIRNA == 'Y']
print('mite_21 ' + len(df_mite_21.index))

In [ ]:
#all miRNA and MITEs

In [ ]:
yeses = 0
stuff = []
total = 0
for k, v in df_all_all.iterrows():
    total += 1
    seqname = v['#Locus'].split(':')[0]
    start = v['#Locus'].split(':')[1].split('-')[0]
    end = v['#Locus'].split(':')[1].split('-')[1]
    ss = False
    for k2, v2 in df_mite_all.iterrows():
        seqname2 = v2['#Locus'].split(':')[0]
        if seqname2 != seqname:
            continue
        start2 = v2['#Locus'].split(':')[1].split('-')[0]
        end2 = v2['#Locus'].split(':')[1].split('-')[1]
        if int(start) <= int(end2) and int(end) >= int(start2):
            stuff_ = {}
            stuff_['miRNA_cluster_locus'] = v['#Locus']
            stuff_['miRNA_cluster_name'] = v['Name']
            stuff_['MITE_miRNA_cluster_locus'] = v2['#Locus']
            stuff_['MITE_miRNA_cluster_name'] = v2['Name']
            stuff.append(stuff_)
            ss = True
            break
    if ss:
        yeses += 1
yeses * 100 / total

In [ ]:
#21nt miRNA and MITEs

In [ ]:
yeses = 0
stuff = []
total = 0
for k, v in df_all_21.iterrows():
    total += 1
    seqname = v['#Locus'].split(':')[0]
    start = v['#Locus'].split(':')[1].split('-')[0]
    end = v['#Locus'].split(':')[1].split('-')[1]
    ss = False
    for k2, v2 in df_mite_21.iterrows():
        seqname2 = v2['#Locus'].split(':')[0]
        if seqname2 != seqname:
            continue
        start2 = v2['#Locus'].split(':')[1].split('-')[0]
        end2 = v2['#Locus'].split(':')[1].split('-')[1]
        if int(start) <= int(end2) and int(end) >= int(start2):
            stuff_ = {}
            stuff_['miRNA_cluster_locus'] = v['#Locus']
            stuff_['miRNA_cluster_name'] = v['Name']
            stuff_['MITE_miRNA_cluster_locus'] = v2['#Locus']
            stuff_['MITE_miRNA_cluster_name'] = v2['Name']
            stuff.append(stuff_)
            ss = True
            break
    if ss:
        yeses += 1
yeses * 100 / total

In [ ]:
#mirBASE with 21

In [ ]:
records = []
for k,v in df_all_21.iterrows():
    record = SeqRecord(Seq(v.MajorRNA), id=v['#Locus'], description=v['Name'])
    records.append(record)
SeqIO.write(records, path_mite_21_seqs, 'fasta')

In [ ]:
df_mirbase = pd.read_csv(path_mirbase, sep="\t")
cols = ['qseqid','sseqid','qstart','qend','sstart','send','mismatch','gaps','pident','evalue','length','qlen','slen','qcovs']
df_mirbase.columns = cols
print(len(df_mirbase.index))
df_mirbase.head(2)

In [ ]:
df_mirbase = df_mirbase[df_mirbase.length >= df_mirbase.qlen - 2]
print('lenght >= length -2 ' + len(df_mirbase.index))

In [ ]:
df_mirbase = df_mirbase[df_mirbase.mismatch + df_mirbase.gaps <= 2]
print('gaps <= 2 ' + len(df_mirbase.index))

In [ ]:
print('MITEs with miRBASE results ' + len(df_mirbase.qseqid.unique().tolist()))

In [ ]:
df_mirbase_s = df_mirbase.groupby('qseqid')['sseqid'].apply(lambda x: "%s" % ', '.join(x))

In [ ]:
df_mirbase_s

In [ ]:
#add miRBASE annotation to df_mite_21

In [ ]:
df_mite_21['miRBASE'] = ''
for k_mite, v_mite in df_mite_21.iterrows():
    for k,v in df_mirbase_s.iterrows():
        if v_mite['#Locus'] == v.qseqid:
            df_mite_21.at[k_mite, 'miRBASE'] += v.sseqid + '.'

In [ ]:
#genomic annotation

In [ ]:
df_ann = pd.read_csv(file_ann, index_col=False, sep='\t', comment='#', header=None)
df_ann.columns = ['seqname' , 'source' , 'feature' , 'start' , 'end' , 'score' , 'strand' , 'frame' , 'attribute']
print('all features: ' + len(df_ann.index))

In [ ]:
df_ann = df_ann[
    (df_ann.feature == 'intron') | 
    (df_ann.feature == 'exon') | 
    (df_ann.feature == 'three_prime_UTR') | 
    (df_ann.feature == 'five_prime_UTR')]
print('gene-related features: ' + len(df_ann.index))

In [ ]:
df_ann_promoters = pd.read_csv(file_ann_promoters, index_col=False, sep='\t', comment='#', header=None)
df_ann_promoters.columns = ['seqname' , 'source' , 'feature' , 'start' , 'end' , 'score' , 'strand' , 'frame' , 'attribute']
print('promoters: ' + len(df_ann_promoters.index))

In [ ]:
df_ann_total = pd.concat([df_ann, df_ann_promoters])
print('total: ' + len(df_ann_total.index))

In [ ]:
df_mite_21['feature'] = ''
df_mite_21['gene'] = ''
for k_mite, v_mite in df_mite_21.iterrows():
    cur = df_ann_total[df_ann_total.seqname == v_mite.seqname]
    for k_feature,v_feature in cur.iterrows():
        start = int(v_feature.start)
        end = int(v_feature.end)
        #overlaps with gene
        if int(v_mite.end) >= start and int(v_mite.start) <= end:
            df_mite_21.at[k_mite, 'feature'] += v_feature.feature + '.'
            df_mite_21.at[k_mite, 'gene'] += v_feature.attribute + '.'
            print (v_mite.attribute, v_feature.feature)
            break